# PREAMBLE
<script
  src="https://cdn.mathjax.org/mathjax/latest/MathJax.js?config=TeX-AMS-MML_HTMLorMML"
  type="text/javascript">
</script>

In [1]:
import numpy as np
import pandas as pd
import numpy.linalg as la
import seaborn as sns
sns.set()
sns.set_style("ticks")
from validphys.api import API
from validphys.loader import Loader
from matplotlib import pyplot as plt
from matplotlib import cm
l = Loader()
import yaml

# Definition of the input

In [2]:
fit="221207-ern-002"

theory_max = 236
theory_mid = 205
theory_min = 201

alphas_step_size = 0.003
alphas_central = 0.119

covmat_scaling_factor = 1

# COMPUTATION OF $\alpha_s$

In [3]:
fitpath = API.fit(fit=fit).path 
filterpath = fitpath / 'filter.yml'
import yaml
with open(filterpath) as f:
    filterfile = yaml.safe_load(f)
pdf_ori=filterfile['theorycovmatconfig']['pdf']

In [4]:
common_dict = dict(dataset_inputs={"from_": "fit"},
            fit=fit,
            use_cuts="fromfit",
            metadata_group="nnpdf31_process",)

In [5]:
#Inputs for central theory
inps_central = dict(theoryid=theory_mid,pdf=pdf_ori,**common_dict)

In [6]:
#Inputs for plus theory
inps_plus = dict(theoryid=theory_max,pdf=pdf_ori,**common_dict)

In [7]:
#Inputs for minus theory
inps_minus = dict(theoryid=theory_min,pdf=pdf_ori,**common_dict)

In [8]:
#Inputs for central theory
inps_central_fit = dict(theoryid=theory_mid,pdf={"from_": "fit"},**common_dict)

In [9]:
#Experimental covariance matrix
C = API.groups_covmat(**inps_central)

In [10]:
dsindex=API.groups_index(**inps_central)

In [11]:
datth_central = API.group_result_table_no_table(**inps_central)


-- Reading COMMONDATA for Dataset: ATLASPHT15_SF
nData: 53 nSys: 2
-- COMMONDATA Files for ATLASPHT15_SF successfully read.

LHAPDF 6.4.0 loading all 101 PDFs in set 221206-ern-002
221206-ern-002, version 1; 101 PDF members

-- Reading COMMONDATA for Dataset: ATLAS_1JET_8TEV_R06_DEC
nData: 171 nSys: 677
-- COMMONDATA Files for ATLAS_1JET_8TEV_R06_DEC successfully read.


-- Reading COMMONDATA for Dataset: CMS_1JET_8TEV
nData: 239 nSys: 293
-- COMMONDATA Files for CMS_1JET_8TEV successfully read.


-- Reading COMMONDATA for Dataset: ATLAS_2JET_7TEV_R06
nData: 90 nSys: 474
-- COMMONDATA Files for ATLAS_2JET_7TEV_R06 successfully read.


-- Reading COMMONDATA for Dataset: CMS_2JET_7TEV
nData: 54 nSys: 88
-- COMMONDATA Files for CMS_2JET_7TEV successfully read.


-- Reading COMMONDATA for Dataset: CHORUSNUPb_dw_ite
nData: 607 nSys: 1014
-- COMMONDATA Files for CHORUSNUPb_dw_ite successfully read.


-- Reading COMMONDATA for Dataset: CHORUSNBPb_dw_ite
nData: 607 nSys: 114
-- COMMONDATA Fil

In [12]:
datth_plus = API.group_result_table_no_table(**inps_plus)

In [13]:
datth_minus = API.group_result_table_no_table(**inps_minus)

In [14]:
datth_central_fit = API.group_result_table_no_table(**inps_central_fit)

LHAPDF 6.4.0 loading all 501 PDFs in set 221207-ern-002
221207-ern-002, version 1; 501 PDF members


In [15]:
dat_central=datth_central["data_central"]
th_replicas_fit=datth_central_fit.iloc[:,2:].to_numpy()

Computation of Eqs.(3.37)-(3.38) in [arXiv:2105.05114](https://arxiv.org/pdf/2105.05114.pdf)

In [16]:
beta_tilde = np.sqrt(covmat_scaling_factor)*(alphas_step_size/np.sqrt(2))*np.array([1,-1])
S_tilde = beta_tilde@beta_tilde

In [17]:
delta_plus  = (np.sqrt(covmat_scaling_factor)/np.sqrt(2))*(datth_plus["theory_central"] - datth_central["theory_central"]).to_numpy()
delta_minus = (np.sqrt(covmat_scaling_factor)/np.sqrt(2))*(datth_minus["theory_central"] - datth_central["theory_central"]).to_numpy()
beta = [delta_plus,delta_minus]
S_hat = beta_tilde@beta

In [18]:
S = np.outer(delta_plus,delta_plus)+np.outer(delta_minus,delta_minus)
S = pd.DataFrame(S,index=dsindex,columns=dsindex)
S = pd.DataFrame(S.values, index=C.index, columns=C.index)

In [19]:
invcov = la.inv(C+S)

In [20]:
# Different from the prediction of the mean PDF (i.e. replica0)
mean_prediction = np.mean(th_replicas_fit[:],axis=1)

In [21]:
X = np.zeros_like(C.values)
for i in range(th_replicas_fit.shape[1]):
    X += np.outer((th_replicas_fit[:,i]-mean_prediction),(th_replicas_fit[:,i]-mean_prediction))
X *= 1/th_replicas_fit.shape[1]

Final result

In [22]:
# BUG: dat_central should become average over data replicas
delta_T_tilde = S_hat@invcov@(dat_central-mean_prediction)
P_tilde = S_hat.T@invcov@X@invcov@S_hat + (S_tilde - S_hat.T@invcov@S_hat)
pred = alphas_central + delta_T_tilde
unc  = np.sqrt(P_tilde)

In [23]:
aa = pd.read_csv(
    fitpath / 'tables/datacuts_theory_theorycovmatconfig_theory_covmat_custom.csv', 
    sep='\t',encoding='utf-8', index_col=2,header=3,skip_blank_lines=False,
);
if np.allclose(S.to_numpy(),aa.to_numpy()[:,2:].astype('float64')):
    print(rf"Prediction for $\alpha_s$: {pred:.5f} +/- {unc:.5f}")
else:
    print("Reconstructed theory covmat, S, is note the same as the stored covmat!")

Prediction for $\alpha_s$: 0.12005 +/- 0.00036
